In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable

In [3]:
#导入数据
factor_df=pd.read_csv("/data01/data_for_intern/factor_data_13_19.csv")
return_df=pd.read_csv("/data01/data_for_intern/return_data_13_19.csv")
return_label=return_df.drop(columns=['return_adj_d001','return_adj_d005','car_hs300_b30_d005'])
return_label['car_hs300_b30_d001']=(return_label['car_hs300_b30_d001']>=0)*1
#利用内连接方式合并数据
merged_factor_return = factor_df.merge(return_label,on=["stock_code","trade_date"],how="inner")
merged_factor_return

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41,car_hs300_b30_d001
0,000001.SZ,2013-01-04,0.007966,0.024510,0.032475,-8.0,-0.114170,0.03,0.440203,-0.376429,...,0.790698,0.724551,0.812212,20.869800,43.5275,0.741100,0.680235,1.57469,3.43924,1
1,000001.SZ,2013-01-07,0.023154,0.006258,0.029412,-7.0,-0.589069,-0.31,0.129240,-0.840631,...,0.856688,0.909091,0.841909,18.507200,41.6409,0.741100,0.704779,1.47261,3.15030,0
2,000001.SZ,2013-01-08,0.004295,0.026994,0.031288,-5.0,-0.150607,0.30,0.957488,-0.160806,...,0.676724,0.727273,0.786972,20.082300,29.5337,0.605634,0.705782,1.48008,3.02077,0
3,000001.SZ,2013-01-09,0.003759,0.010025,0.013784,-3.0,-0.836842,0.14,0.802262,-0.532596,...,0.632231,0.642424,0.759596,8.662940,27.9438,0.536232,0.699468,1.47922,3.01475,0
4,000001.SZ,2013-01-10,0.015132,0.004414,0.019546,-1.0,-0.565992,-0.01,0.452223,-0.712085,...,0.629167,0.516667,0.628788,12.206900,27.2890,0.520000,0.690864,1.51058,3.06918,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088415,603999.SH,2019-12-13,0.063063,0.007508,0.070571,-9.0,-0.875646,-0.32,0.138746,-0.876772,...,0.542857,0.855769,0.822900,1.148900,20.9302,0.029940,0.307711,1.14262,3.09775,1
5088416,603999.SH,2019-12-16,0.038682,0.007163,0.045845,-9.0,-0.880784,-0.18,0.306574,-0.751634,...,0.583333,0.884298,0.811730,0.782227,27.0718,0.187879,0.276938,1.04110,2.77682,0
5088417,603999.SH,2019-12-17,0.002821,0.016925,0.019746,-8.0,-0.854110,0.11,0.863933,-0.479172,...,0.654971,0.778761,0.839609,0.342224,29.0801,0.298013,0.260858,1.04584,2.66659,0
5088418,603999.SH,2019-12-18,0.010029,0.012894,0.022923,-7.0,-0.733061,0.04,0.511944,-0.716503,...,0.670659,0.736364,0.799807,0.391114,29.2537,0.324324,0.252109,1.07986,2.67018,1


In [4]:
#筛选最近一个月的data
merged_factor_return[(merged_factor_return["stock_code"]=="000002.SZ") & (merged_factor_return["trade_date"]=="2019-11-01")]
#从"2019-11-01"的数据开始清洗作为training_set

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41,car_hs300_b30_d001
3353,000002.SZ,2019-11-01,0.011295,0.013931,0.025226,-1.0,-0.203252,-0.3,0.145479,-0.686855,...,0.46402,0.407643,0.211796,92.4158,9.70874,0.35743,0.24854,0.352462,-0.146485,0


In [5]:
merged_factor_return[(merged_factor_return["stock_code"]=="000002.SZ") & (merged_factor_return["trade_date"]=="2019-12-02")]
#留12月的数据作为validation_set,"2019-12-02"开始清洗

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41,car_hs300_b30_d001
3374,000002.SZ,2019-12-02,0.00897,0.000359,0.009329,-7.0,-0.131487,-0.35,0.068559,-0.854795,...,0.714286,0.711111,0.642946,57.9323,42.9224,0.431472,0.337125,2.36833,8.71862,0


In [6]:
idx_name=np.unique(merged_factor_return["stock_code"])
idx_name

array(['000001.SZ', '000002.SZ', '000004.SZ', ..., '603997.SH',
       '603998.SH', '603999.SH'], dtype=object)

In [7]:
temp=merged_factor_return[(merged_factor_return["stock_code"]=="000001.SZ")].drop(['car_hs300_b30_d001'],axis=1)
d1=merged_factor_return[(merged_factor_return["stock_code"]=="000001.SZ") & (merged_factor_return["trade_date"]=="2019-11-01")].index
d1=int(np.array(d1))
d2=merged_factor_return[(merged_factor_return["stock_code"]=="000001.SZ") & (merged_factor_return["trade_date"]=="2019-11-29")].index
d2=int(np.array(d2))
temp=temp.drop(["stock_code","trade_date"],axis=1)
train_data=temp.loc[d1:d1+7].values.reshape(1,8,42)
valid_data=temp.loc[d2+1:d2+8].values.reshape(1,8,42)

In [8]:
(temp.loc[d1+25:d1+33]).shape,d1,d2

((9, 42), 1659, 1679)

In [9]:
df_data=merged_factor_return.drop(['car_hs300_b30_d001'],axis=1)

In [10]:
waste=[]
for name in idx_name:
    temp=df_data[(df_data["stock_code"]==name)]
    d1=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]=="2019-11-01")].index
    if d1.size==0:
        waste.append(name)
        continue
    d1=int(np.array(d1))
    d2=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]=="2019-12-02")].index
    if d2.size==0:
        waste.append(name)
        continue
    d2=int(np.array(d2))
    temp=temp.drop(["stock_code","trade_date"],axis=1)
    for i in range(d1,d2-7):
        train_temp=temp.loc[i:i+7].values.reshape(1,8,42)
        train_data=np.concatenate((train_data,train_temp),axis=0)

In [11]:
valid_data.shape

(1, 8, 42)

In [12]:
waste2=[]
for name in idx_name:
    temp=df_data[(df_data["stock_code"]==name)]
    d1=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]=="2019-12-02")].index
    if d1.size==0:
        waste2.append(name)
        continue
    d1=int(np.array(d1))
    d2=df_data[(df_data["stock_code"]==name) & (df_data["trade_date"]=="2019-12-19")].index
    if d2.size==0:
        waste2.append(name)
        continue
    d2=int(np.array(d2))
    temp=temp.drop(["stock_code","trade_date"],axis=1)
    for i in range(d1,d2-7):
        valid_temp=temp.loc[i:i+7].values.reshape(1,8,42)
        valid_data=np.concatenate((valid_data,valid_temp),axis=0)

In [13]:
clean_name_valid = np.setdiff1d(idx_name,waste2)
clean_name_valid

array(['000001.SZ', '000002.SZ', '000004.SZ', ..., '603997.SH',
       '603998.SH', '603999.SH'], dtype=object)

In [14]:
np.save('wave_train_data',train_data)
np.save('wave_valid_data',valid_data)

In [16]:
label=merged_factor_return[["car_hs300_b30_d001","stock_code","trade_date"]]
#label

In [17]:
d1=label[(label["stock_code"]=="000001.SZ") & (label["trade_date"]=="2019-11-01")].index
d1=int(np.array(d1))
d2=label[(label["stock_code"]=="000001.SZ") & (label["trade_date"]=="2019-11-29")].index
d2=int(np.array(d2))
temp=label.drop(["stock_code","trade_date"],axis=1)
train_label=temp.loc[d1+8].values
valid_label=temp.loc[d2+9].values

In [18]:
train_label,valid_label

(array([0]), array([1]))

In [19]:
#train_label
waste=[]
for name in idx_name:
    temp=label[(label["stock_code"]==name)]
    d1=label[(label["stock_code"]==name) & (label["trade_date"]=="2019-11-01")].index
    if d1.size==0:
        waste.append(name)
        continue
    d1=int(np.array(d1))
    d2=label[(label["stock_code"]==name) & (label["trade_date"]=="2019-12-02")].index
    if d2.size==0:
        waste.append(name)
        continue
    d2=int(np.array(d2))
    temp=temp.drop(["stock_code","trade_date"],axis=1)
    for i in range(d1,d2-7):
        train_temp=temp.loc[i+8].values
        train_label=np.concatenate((train_label,train_temp),axis=0)

In [20]:
#valid_label
waste=[]
for name in idx_name:
    temp=label[(label["stock_code"]==name)]
    d1=label[(label["stock_code"]==name) & (label["trade_date"]=="2019-12-02")].index
    if d1.size==0:
        waste.append(name)
        continue
    d1=int(np.array(d1))
    d2=label[(label["stock_code"]==name) & (label["trade_date"]=="2019-12-19")].index
    if d2.size==0:
        waste.append(name)
        continue
    d2=int(np.array(d2))
    temp=temp.drop(["stock_code","trade_date"],axis=1)
    for i in range(d1,d2-7):
        valid_temp=temp.loc[i+8].values
        valid_label=np.concatenate((valid_label,valid_temp),axis=0)

In [21]:
#np.save('train_data',train_data)
#np.save('valid_data',valid_data)
np.save('wave_train_label',train_label)
np.save('wave_valid_label',valid_label)